## General

In [1]:
import os
from Crypto.Protocol.KDF import PBKDF2 
from Crypto.Cipher import AES
from pynq import Overlay
from pynq import allocate
import numpy as np
import time
import gc

In [2]:
def prefix(num, decimal=2):
    if decimal<0:
        strformat = '%f'
    else:
        strformat = '%%.%df'%(decimal)
    if num>=1000000000000:
        return (strformat+' T')%(num/1000000000000)
    elif num>=1000000000:
        return (strformat+' G')%(num/1000000000)
    elif num>=1000000:
        return (strformat+' M')%(num/1000000)
    elif num>=1000:
        return (strformat+' k')%(num/1000)
    else:
        return '%s'%(num)

In [3]:
def pass_fail(condition):
    return 'PASS' if condition else 'FAILED'

###### Mount USB drive

In [4]:
%run ./linux_usb_mount.ipynb

/mnt/usb


######  Generate an AES key

In [5]:
#AES Utils
%run ./AES_Utils.ipynb

In [6]:
print(key)

b'\x97\xc6\x17\x85\xe7(\xa6\x01\xb8\xd7\x1d\xa4L\xb3\x0b\x1c#\xff\xfem\x12\x9d%\xcc_\xd7hVb\\\x8d\xfa'


In [7]:
%run ./AES_Cores.ipynb

In [8]:
# Testing parameters

In [9]:
# TEST_GB_001 TEST_GB_002
# TEST_MB_001 TEST_MB_005 TEST_MB_010 TEST_MB_050 TEST_MB_100 TEST_MB_500
# TEST_kB_001 TEST_kB_005 TEST_kB_010 TEST_kB_050 TEST_kB_100 TEST_kB_500

In [10]:
encrypt_file_source = os.path.join(usb_path+'/testFile/', "TEST_MB_050")

In [11]:
encrypt_file_rename = encrypt_file_source+".rename"

## Software

###### Configuration

In [12]:
skip_software = False

###### Free memory

In [13]:
gc.collect()

120

###### Encrypt / Decrypt file

In [14]:
encrypt_extention = '.ECRYP'

In [15]:
def encryptFile(fileName, key, rename=None, batch_size_log2=24):
    batch_size = int(2**batch_size_log2)
    get_file_size = os.path.getsize(fileName)
    file_size = get_file_size
    if rename is None:
        rename = fileName+encrypt_extention
    try:
        fr = open(fileName, 'rb')
        fw = open(rename, 'wb')
        #fw.write(b'ECRYP')
        #fw.write(bytes([batch_size_log2]))
        
        while file_size>0:
            content = fr.read(batch_size)
            if file_size <= batch_size:
                content = aesEncrypt(content, key, padding=True)
            else:
                content = aesEncrypt(content, key, padding=False)
            fw.write(content)
            file_size = file_size-batch_size
    finally:
        if fr:
            fr.close()
        if fw:
            fw.close()
    
    return get_file_size

In [16]:
def decryptFile(fileName, key, rename=None, batch_size_log2=24):
    if encrypt_extention == fileName[-len(encrypt_extention):]:
        get_file_size = os.path.getsize(fileName)
        file_size = get_file_size
        if rename is None:
            rename = fileName[:-len(encrypt_extention)]
        try:
            fr = open(fileName, 'rb')
            #if fr.read(5) != b'ECRYP':
            #    fr.close()
            #    return False
            #batch_size_log2 = int.from_bytes(fr.read(1), byteorder='big')

            batch_size = 2**batch_size_log2
            fw = open(rename, 'wb')

            while file_size>0:
                content = fr.read(batch_size)
                if file_size <= batch_size:
                    content = aesDecrypt(content, key, padding=True)
                else:
                    content = aesDecrypt(content, key, padding=False)
                fw.write(content)
                file_size = file_size-batch_size
        finally:
            if fr:
                fr.close()
            if fw:
                fw.close()
                
        return True
    else:
        return False

###### Testing

In [17]:
print("Encrypting: %s"%(encrypt_file_source))

sum_se_filesize = os.path.getsize(encrypt_file_source)
print("File Size: %sB"%(prefix(sum_se_filesize, decimal=2)))

time_start = time.time()
if not skip_software:
    encryptFile(encrypt_file_source, key, rename = encrypt_file_rename+encrypt_extention)
time_stop = time.time()

sum_se_duration = time_stop - time_start
print("Duration: %f sec"%(sum_se_duration))

print("Encrypt done!\n")

Encrypting: /mnt/usb/testFile/TEST_MB_050
File Size: 52.43 MB
Duration: 8.905452 sec
Encrypt done!



In [18]:
print("Decrypting: %s"%(encrypt_file_rename+encrypt_extention))

if not skip_software:
    sum_sd_filesize = os.path.getsize(encrypt_file_rename+encrypt_extention)
else:
    sum_sd_filesize = 0
print("File Size: %sB"%(prefix(sum_se_filesize, decimal=2)))

time_start = time.time()
if not skip_software:
    decryptFile(encrypt_file_rename+encrypt_extention, key)
time_stop = time.time()

sum_sd_duration = time_stop - time_start
print("Duration: %f sec"%(sum_sd_duration))

print("Decrypt done!\n")

Decrypting: /mnt/usb/testFile/TEST_MB_050.rename.ECRYP
File Size: 52.43 MB
Duration: 7.815938 sec
Decrypt done!



###### Checksum

In [19]:
if not skip_software:
    sum_so_md5 = cmd("md5sum %s"%(encrypt_file_source))
    print(sum_so_md5)
    sum_so_md5 = sum_so_md5.split(" ")[0]
    sum_se_md5 = cmd("md5sum %s"%(encrypt_file_rename+encrypt_extention))
    print(sum_se_md5)
    sum_se_md5 = sum_se_md5.split(" ")[0]
    sum_sd_md5 = cmd("md5sum %s"%(encrypt_file_rename))
    print(sum_sd_md5)
    sum_sd_md5 = sum_sd_md5.split(" ")[0]
else:
    sum_so_md5='SKIP-000'
    sum_se_md5='SKIP-001'
    sum_sd_md5='SKIP-002'

d23ecd9aa176ee800ae0b73d383dcc75  /mnt/usb/testFile/TEST_MB_050

e9778ec68a92e52fb7c64deea39d03bd  /mnt/usb/testFile/TEST_MB_050.rename.ECRYP

d23ecd9aa176ee800ae0b73d383dcc75  /mnt/usb/testFile/TEST_MB_050.rename



###### Delete files

In [20]:
print(cmd("rm -f %s"%(encrypt_file_rename+encrypt_extention)))
print(cmd("rm -f %s"%(encrypt_file_rename)))

## Hardware

###### Configuration

In [21]:
hardware_encrypt_overlay = "./bit/encrypt_aes256_150M_core6.bit"
hardware_decrypt_overlay = "./bit/decrypt_aes256_150M_core6.bit"
hardware_core_count = 6
hardware_block_size = 2**23
hardware_self_test = True

###### Free memory

In [22]:
gc.collect()

0

#### Encrypt file

In [23]:
ol = Overlay(hardware_encrypt_overlay)

In [24]:
core_number = hardware_core_count
block_size = hardware_block_size

cores = [ AES_Core( 
    ol.__getattr__(f"AES_ECB_encrypt_{i}") ,
    ol.__getattr__(f"axi_dma_write_{i}") ,
    ol.__getattr__(f"axi_dma_read_{i}"),
    key ,
    block_size,
    self_test = hardware_self_test
) for i in range(core_number)]

AES_ECB_encrypt_0 self encrypt test success
AES_ECB_encrypt_1 self encrypt test success
AES_ECB_encrypt_2 self encrypt test success
AES_ECB_encrypt_3 self encrypt test success
AES_ECB_encrypt_4 self encrypt test success
AES_ECB_encrypt_5 self encrypt test success


In [25]:
aes_hw = AES_Schedule( cores,  block_size = block_size )

Init success with 6 AES cores


###### Testing

In [26]:
sum_he_filesize = os.path.getsize(encrypt_file_source)
print("File Size: %sB"%(prefix(sum_he_filesize, decimal=2)))
start_time = time.time()
aes_hw.enqueue_file(file=encrypt_file_source)
aes_hw.start_encrypt_schedule()
end_time = time.time()
sum_he_duration = end_time - start_time
print(f"time: {sum_he_duration}")

File Size: 52.43 MB
start encrypt /mnt/usb/testFile/TEST_MB_050 with size 52428800
need do 6 time
Start part    6 /    6 on core #5
time: 4.512328624725342


###### Free memory

In [27]:
del ol
del aes_hw
gc.collect()

60

#### Decrypt file

In [28]:
ol = Overlay(hardware_decrypt_overlay)

In [29]:
core_number = hardware_core_count
block_size = hardware_block_size

for i,core in enumerate(cores):
    core.init(
        ol.__getattr__(f"AES_ECB_decrypt_{i}") ,
        ol.__getattr__(f"axi_dma_write_{i}") ,
        ol.__getattr__(f"axi_dma_read_{i}"),
        key ,
        self_test = hardware_self_test
    )

AES_ECB_decrypt_0 self decrypt test success
AES_ECB_decrypt_1 self decrypt test success
AES_ECB_decrypt_2 self decrypt test success
AES_ECB_decrypt_3 self decrypt test success
AES_ECB_decrypt_4 self decrypt test success
AES_ECB_decrypt_5 self decrypt test success


In [30]:
aes_hw = AES_Schedule( cores,  block_size = block_size )

Init success with 6 AES cores


###### Testing

In [31]:
sum_hd_filesize = os.path.getsize(encrypt_file_source)
print("File Size: %sB"%(prefix(sum_hd_filesize, decimal=2)))
start_time = time.time()
aes_hw.enqueue_file(file=encrypt_file_source+'.encrypt')
aes_hw.start_decrypt_schedule()
end_time = time.time()
sum_hd_duration = end_time - start_time
print(f"time: {sum_hd_duration}")

File Size: 52.43 MB
start decrypt /mnt/usb/testFile/TEST_MB_050.encrypt with size 52428816
need do 6 time
Start part    6 /    6 on core #5
time: 5.3055737018585205


###### Checksum

In [32]:
sum_ho_md5 = cmd("md5sum %s"%(encrypt_file_source))
print(sum_ho_md5)
sum_ho_md5 = sum_ho_md5.split(" ")[0]
sum_he_md5 = cmd("md5sum %s"%(encrypt_file_source+'.encrypt'))
print(sum_he_md5)
sum_he_md5 = sum_he_md5.split(" ")[0]
sum_hd_md5 = cmd("md5sum %s"%(encrypt_file_source+'.encrypt'+'.decrypt'))
print(sum_hd_md5)
sum_hd_md5 = sum_hd_md5.split(" ")[0]

d23ecd9aa176ee800ae0b73d383dcc75  /mnt/usb/testFile/TEST_MB_050

e9778ec68a92e52fb7c64deea39d03bd  /mnt/usb/testFile/TEST_MB_050.encrypt

d23ecd9aa176ee800ae0b73d383dcc75  /mnt/usb/testFile/TEST_MB_050.encrypt.decrypt



###### Delete files

In [33]:
print(cmd("rm -f %s"%(encrypt_file_source+'.encrypt')))
print(cmd("rm -f %s"%(encrypt_file_source+'.encrypt'+'.decrypt')))

###### Free memory

In [34]:
del ol
del aes_hw
gc.collect()

148

## =========================================

## Summary

#### Information

In [35]:
print('%-20s %-80s'%('Target File', encrypt_file_source))
print('%-20s %-80s'%('File Size', prefix(os.path.getsize(encrypt_file_source),decimal=-1)+'B'))

Target File          /mnt/usb/testFile/TEST_MB_050                                                   
File Size            52.428800 MB                                                                    


#### Performance

In [36]:
print('%-20s %-15s %-15s %-15s %-15s'%('', 'Software EN', 'Software DE', 'Hardware EN', 'Hardware DE'))
print('%-20s %-15s %-15s %-15s %-15s'%('Handle File Size', prefix(sum_se_filesize,decimal=-1)+'B', prefix(sum_sd_filesize,decimal=-1)+'B', prefix(sum_he_filesize,decimal=-1)+'B', prefix(sum_hd_filesize,decimal=-1)+'B'))
print('%-20s %-15.4f %-15.4f %-15.4f %-15.4f'%('Duration', sum_se_duration, sum_sd_duration, sum_he_duration, sum_hd_duration))
print('%-20s %-15s %-15s %-15s %-15s'%('Bytes/s', prefix(sum_se_filesize/sum_se_duration)+'B/s', prefix(sum_sd_filesize/sum_sd_duration)+'B/s', prefix(sum_he_filesize/sum_he_duration)+'B/s', prefix(sum_hd_filesize/sum_hd_duration)+'B/s'))

                     Software EN     Software DE     Hardware EN     Hardware DE    
Handle File Size     52.428800 MB    52.428816 MB    52.428800 MB    52.428800 MB   
Duration             8.9055          7.8159          4.5123          5.3056         
Bytes/s              5.89 MB/s       6.71 MB/s       11.62 MB/s      9.88 MB/s      


#### Correctness

In [37]:
print('%-15s %-35s %-35s %-7s'%('MD5 Checksum', 'Software', 'Hardware', ''))
print('%-15s %-35s %-35s %-7s'%('Source File', sum_so_md5, sum_ho_md5, pass_fail(sum_so_md5==sum_ho_md5)))
print('%-15s %-35s %-35s %-7s'%('Encrypted', sum_se_md5, sum_he_md5, pass_fail(sum_se_md5==sum_he_md5)))
print('%-15s %-35s %-35s %-7s'%('Recovery', sum_sd_md5, sum_hd_md5, pass_fail(sum_sd_md5==sum_hd_md5)))
print('%-15s %-35s %-35s %-7s'%('', pass_fail(sum_so_md5==sum_sd_md5), pass_fail(sum_ho_md5==sum_hd_md5), ''))

MD5 Checksum    Software                            Hardware                                   
Source File     d23ecd9aa176ee800ae0b73d383dcc75    d23ecd9aa176ee800ae0b73d383dcc75    PASS   
Encrypted       e9778ec68a92e52fb7c64deea39d03bd    e9778ec68a92e52fb7c64deea39d03bd    PASS   
Recovery        d23ecd9aa176ee800ae0b73d383dcc75    d23ecd9aa176ee800ae0b73d383dcc75    PASS   
                PASS                                PASS                                       


## =========================================